In [1]:
print("hello")

hello


In [2]:
import os
import trimesh
import meshplot as mp
import numpy as np
import torch
from torch import Tensor
from typing import *
from mpm_pytorch import MPMSolver, set_boundary_conditions, get_constitutive
from tqdm.notebook import tqdm  
from functools import partial

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from scipy import ndimage
from scipy.ndimage import gaussian_filter
from skimage import measure
import open3d as o3d
import gc



Warp 0.6.1 initialized:
   CUDA Toolkit: 11.5, Driver: 12.2
   Devices:
     "cpu"    | x86_64
     "cuda:0" | NVIDIA GeForce RTX 3090 (sm_86)
     "cuda:1" | NVIDIA GeForce RTX 3090 (sm_86)
   Kernel cache: /home/cx2219/.cache/warp/0.6.1
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def tensor_size_in_MB(tensor):
    return tensor.element_size() * tensor.nelement() / 1e6  # in MB

def list_gpu_tensors():
    all_tensors = []
    total_tensor_mem = 0.0

    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                if obj.is_cuda:
                    size_mb = tensor_size_in_MB(obj)
                    total_tensor_mem += size_mb
                    all_tensors.append((size_mb, obj.shape, type(obj), obj.device))
        except Exception:
            pass  # ignore inaccessible objects

    # Sort by size
    all_tensors.sort(reverse=True, key=lambda x: x[0])

    print(f"{'Size (MB)':>10} | {'Shape':>20} | {'Type':>30} | {'Device':>10}")
    print("-" * 80)
    for size_mb, shape, typ, device in all_tensors:
        print(f"{size_mb:10.2f} | {str(shape):>20} | {str(typ):>30} | {str(device):>10}")

    print("\n")
    print(f"🧠 Total memory of live tensors: {total_tensor_mem:.2f} MB")
    print(f"📦 torch.cuda.memory_allocated(): {torch.cuda.memory_allocated() / 1e6:.2f} MB")
    print(f"🔒 torch.cuda.memory_reserved():  {torch.cuda.memory_reserved() / 1e6:.2f} MB")

In [4]:
def visualize_frames(
    frames: List[np.ndarray], 
    export_path: str, 
    center: List[float] = [0.5, 0.5, 0.5],
    size: List[float] = [2.0, 2.0, 2.0],
    c: str = 'blue',
    s: float = 20,
    fps: int = 30,
    sample_rate: int = 1
): 
    xlim = [center[0] - size[0] / 2, center[0] + size[0] / 2]
    ylim = [center[1] - size[1] / 2, center[1] + size[1] / 2]
    zlim = [center[2] - size[2] / 2, center[2] + size[2] / 2]
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    scat = ax.scatter([], [], [], s=s)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_zlim(zlim)
    def update(frame):
        print(f"updating the {frame}/{len(frames)} frame")
        ax.cla()
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        ax.set_zlim(zlim)
        scat = ax.scatter(frames[frame][:, 0], frames[frame][:, 1], frames[frame][:, 2], s=s, c=c)
        ax.set_title(f'Frame {frame * sample_rate}')
        return scat
    ani = FuncAnimation(fig, update, frames=len(frames), blit=False)
    ani.save(export_path, writer='pillow', fps=fps)
    plt.close()

In [5]:
# parameters
grid_res = 300

output_dir =  "./output"
tag = "jupytertest"

elasticity_type = 'CorotatedElasticity'
plasticity_type = 'DruckerPragerPlasticity'
color = 'blue'

model_path = f"models/cranium_example_ds_{grid_res}.obj"
scale_factor = 0.6

num_frames = 70
steps_per_frame = 10
initial_velocity = [0.0, 0.0, 0.0]
boundary_conditions = [{"type": 'sdf_collider_test',
      "start_time": 0.0,
      "end_time": 1e3}]

sample_rate = 10
obj_name = model_path.split("/")[-1].split(".")[0] + "_" + tag
export_path = os.path.join(output_dir, obj_name + "_deformation.gif")

# Import Point Cloud

In [6]:
mesh = trimesh.load(model_path)

v = np.asarray(mesh.vertices)

#     set_boundary_conditions(mpm_solver, sim_params.boundary_conditions)

# mesh.vertices = o3d.utility.Vector3dVector(v)
# v = np.asarray(mesh.vertices)

# Step 3: Visualize using meshplot
mp.plot(v, shading={"wireframe": False})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3093645…

# create particles

In [7]:
def get_pc_from_v(
    vertices: np.array,
    scale: float,
    center: List[float], 
    device: torch.device = torch.device("cuda")
) -> Tensor:
    points = torch.tensor(vertices, dtype=torch.float32)

    # Scale about center
    center_tensor = torch.tensor(center, dtype=torch.float32)
    points = (points - center_tensor) * scale + center_tensor

    return points.to(device)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
particles = get_pc_from_v(
    vertices = v,
    device = device,
    center=[0.5, 0.5, 0.5], 
    scale=scale_factor
)
n_particles = particles.shape[0]

In [9]:
particles_show = particles.clone()
v = particles_show.cpu().numpy()
mp.plot(v, shading={"wireframe": False})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3856187…

# deform

In [10]:
n_particles = particles.shape[0]
mpm_solver = MPMSolver(
    particles, 
    enable_train=False,
    device=device,
    gravity=[0.0, 0.0, 0.0]
)
# set_boundary_conditions(mpm_solver,boundary_conditions)
elasticity = get_constitutive(elasticity_type, device=device)
plasticity = get_constitutive(plasticity_type, device=device)


# my deformation

In [11]:
def add_pre_particle_process(model: MPMSolver, select:Tensor, direction:list[float], start_time: float =0.0, end_time: float=999.0):
    # select
    def my_deform(model: MPMSolver, x: Tensor, v:Tensor, start_time: float, end_time:float, select: Tensor):

        time = model.time
        unit_displacement = torch.tensor(direction, device=model.device).float()
        unit_displacement = unit_displacement / unit_displacement.norm()
        displacement = 0.000001
        if time >= start_time and time < end_time:
            # x[select] = x[select] + unit_displacement * displacement
            v[select] = v[select] + unit_displacement * displacement/model.dt


    
    model.pre_particle_process.append(
        partial(
            my_deform,
            start_time=start_time,
            end_time=end_time,
            select = select
        )
    )

In [12]:
coords_static = particles.cpu().numpy()  # Extract x-coordinates

x_max = np.max(coords_static[:,0])
x_min = np.min(coords_static[:,0])
y_max = np.max(coords_static[:,1])
y_min = np.min(coords_static[:,1])
z_max = np.max(coords_static[:,2])
z_min = np.min(coords_static[:,2])
print(f"x from {x_min} to {x_max}")
print(f"y from {y_min} to {y_max}")
print(f"z from {z_min} to {z_max}")



x from 0.20200666785240173 to 0.5692307949066162
y from 0.20200666785240173 to 0.7979933023452759
z from 0.20200666785240173 to 0.4809364378452301


In [13]:
def select_particles(particles, point, normal):
    point = torch.tensor(point, device=particles.device, dtype=particles.dtype)
    normal = torch.tensor(normal, device=particles.device, dtype=particles.dtype)

    diff = particles - point  # shape (N, 3)
    dot = torch.matmul(diff, normal)  # (N,)
    
    return dot < 0  # left side of the plane (negative dot product)

In [14]:
# Test points
# Define plane
point = [0.3, 0.3, 0.4]
direction = [-1, 1, -1]

# Apply selector
select = select_particles(particles, point, direction)

# Count
num_total = len(particles)
num_left = select.sum().item()
num_right = num_total - num_left

# Percentage
pct_left = 100.0 * num_left / num_total
pct_right = 100.0 * num_right / num_total

print(f"🔹 Particles on left side:  {num_left}/{num_total} ({pct_left:.2f}%)")
print(f"🔸 Particles on right side: {num_right}/{num_total} ({pct_right:.2f}%)")


color = torch.zeros(len(particles))
color[select] = 0.1  
color = color.cpu().numpy()
mp.plot(v, c = color, shading={"wireframe": False})

🔹 Particles on left side:  183366/1981407 (9.25%)
🔸 Particles on right side: 1798041/1981407 (90.75%)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3856187…

In [15]:
point1 = [0.4, 0.4, 0.3]
direction1 = [1, 1, 1]
point2 = [0.5, 0.6, 0.6]
direction2 = [-1, -1, 0.3]
point3 = [0.3, 0.3, 0.4]
direction3 = [-1, 1, -1]

select1 = select_particles(particles, point1, direction1)
select2 = select_particles(particles, point2, direction2)
select3 = select_particles(particles, point3, direction3)


In [16]:
color = torch.zeros(len(particles))
color[select1] = 0.2  
color[select2] = 0.3
color[select3] = 0.4
color[select1 & select2] = 0.5
color[select1 & select3] = 0.6
color[select2 & select3] = 0.7
color[select1 & select2 & select3] = 0.8


color = color.cpu().numpy()
mp.plot(v, c = color, shading={"wireframe": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3856187…

In [17]:
add_pre_particle_process(mpm_solver, select = select1, direction = direction1)
add_pre_particle_process(mpm_solver, select = select2, direction = direction2)
add_pre_particle_process(mpm_solver, select = select3, direction = direction3)



In [18]:
list_gpu_tensors()

 Size (MB) |                Shape |                           Type |     Device
--------------------------------------------------------------------------------
     23.78 | torch.Size([1981407, 3]) |         <class 'torch.Tensor'> |     cuda:0
     23.78 | torch.Size([1981407, 3]) |         <class 'torch.Tensor'> |     cuda:0
     23.78 | torch.Size([1981407, 3]) |         <class 'torch.Tensor'> |     cuda:0
      1.98 | torch.Size([1981407]) |         <class 'torch.Tensor'> |     cuda:0
      1.98 | torch.Size([1981407]) |         <class 'torch.Tensor'> |     cuda:0
      1.98 | torch.Size([1981407]) |         <class 'torch.Tensor'> |     cuda:0
      1.98 | torch.Size([1981407]) |         <class 'torch.Tensor'> |     cuda:0
      0.19 | torch.Size([15625, 3]) |         <class 'torch.Tensor'> |     cuda:0
      0.19 | torch.Size([15625, 3]) |         <class 'torch.Tensor'> |     cuda:0
      0.06 |  torch.Size([15625]) |         <class 'torch.Tensor'> |     cuda:0
      0.00 |  torch

/home/cx2219/miniconda3/envs/mpmpytorch/lib/python3.9/site-packages/torch/__init__.py:1117: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(obj, torch.Tensor)
/tmp/ipykernel_2135208/2949170866.py:10: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):


In [19]:
# Initialize particle states
x = particles
v = torch.stack([torch.tensor(initial_velocity, device=device) for _ in range(n_particles)])
C = torch.zeros((n_particles, 3, 3), device=device)
F = torch.eye(3, device=device).unsqueeze(0).repeat(n_particles, 1, 1)


In [20]:
# Run simulation
frames = []
for frame in tqdm(range(num_frames), desc='Simulating'):
    frames.append(x.cpu().numpy())
    for step in tqdm(range(steps_per_frame), desc='Step'):
        # Update stress
        stress = elasticity(F)
        # Particle to grid, grid update, grid to particle
        x, v, C, F = mpm_solver(x, v, C, F, stress)
        # Plasticity correction
        F = plasticity(F)

Simulating:   0%|          | 0/70 [00:00<?, ?it/s]

Step:   0%|          | 0/10 [00:00<?, ?it/s]

Module mpm_pytorch.constitutive_models.warp_svd.svd load on device 'cuda:0' took 1.01 ms


Step:   0%|          | 0/10 [00:00<?, ?it/s]

Step:   0%|          | 0/10 [00:00<?, ?it/s]

...

Step:   0%|          | 0/10 [00:00<?, ?it/s]

Step:   0%|          | 0/10 [00:00<?, ?it/s]

Step:   0%|          | 0/10 [00:00<?, ?it/s]

Step:   0%|          | 0/10 [00:00<?, ?it/s]

Step:   0%|          | 0/10 [00:00<?, ?it/s]

In [21]:
list_gpu_tensors()

 Size (MB) |                Shape |                           Type |     Device
--------------------------------------------------------------------------------
     71.33 | torch.Size([1981407, 3, 3]) |         <class 'torch.Tensor'> |     cuda:0
     71.33 | torch.Size([1981407, 3, 3]) |         <class 'torch.Tensor'> |     cuda:0
     71.33 | torch.Size([1981407, 3, 3]) |         <class 'torch.Tensor'> |     cuda:0
     23.78 | torch.Size([1981407, 3]) |         <class 'torch.Tensor'> |     cuda:0
     23.78 | torch.Size([1981407, 3]) |         <class 'torch.Tensor'> |     cuda:0
     23.78 | torch.Size([1981407, 3]) |         <class 'torch.Tensor'> |     cuda:0
     23.78 | torch.Size([1981407, 3]) |         <class 'torch.Tensor'> |     cuda:0
     23.78 | torch.Size([1981407, 3]) |         <class 'torch.Tensor'> |     cuda:0
      1.98 | torch.Size([1981407]) |         <class 'torch.Tensor'> |     cuda:0
      1.98 | torch.Size([1981407]) |         <class 'torch.Tensor'> |     cud

/tmp/ipykernel_2135208/2949170866.py:10: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):


In [22]:
particles_show = x.clone()
v = particles_show.cpu().numpy()
mp.plot(v, c = color, shading={"wireframe": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3503157…

In [23]:
new_frames = frames[::sample_rate]

# Visualize
print(f'Rendering to {export_path}...')
visualize_frames(
    new_frames, 
    export_path=export_path, 
    size=[1, 1, 1], 
    c=color,
    sample_rate = sample_rate
)

Rendering to ./output/cranium_example_ds_300_jupytertest_deformation.gif...
updating the 0/7 frame
updating the 0/7 frame
updating the 1/7 frame
updating the 2/7 frame
updating the 3/7 frame
updating the 4/7 frame
updating the 5/7 frame
updating the 6/7 frame


# import messed up pc to voxel inside

In [24]:

points_inside = particles_show.cpu().numpy()
indices = (points_inside * grid_res).astype(int)
indices = np.clip(indices, 0, grid_res - 1)  # make sure indices are valid

# Create occupancy grid
inside = np.zeros((grid_res, grid_res, grid_res), dtype=bool)
for i, j, k in indices:
    inside[i, j, k] = True

inside = inside.ravel(order='C')

In [25]:
p = mp.plot(points_inside, shading={"wireframe": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3503157…

# voxel to SDF

In [26]:
# continuous
inside_volume = inside.reshape((grid_res, grid_res, grid_res))  # shape: (Z, Y, X)

outside_mask = ~inside_volume  # 0=inside, 1=outside

dist_out = ndimage.distance_transform_edt(outside_mask)

dist_in = ndimage.distance_transform_edt(inside_volume)

sdf_volume = dist_out - dist_in  # outside: positive, inside: negative

sdf_smoothed = gaussian_filter(sdf_volume, sigma=1.0)

sdf_volume = sdf_smoothed


# SDF to Surface Mesh

In [27]:


verts, faces, normals, values = measure.marching_cubes(sdf_volume, level=0.0)

verts = verts[:, [2, 1, 0]]  # 把 zyx 转成 xyz

spacing = 1.0 / (sdf_volume.shape[0] - 1)
verts *= spacing  # 恢复真实坐标

mesh_surface = trimesh.Trimesh(vertices=verts, faces=faces, vertex_normals=normals)


In [28]:

v = np.asarray(mesh_surface.vertices)
f = np.asarray(mesh_surface.faces)
n = np.asarray(mesh_surface.vertex_normals)

mp.plot(v, f, shading={"wireframe": False})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3355654…

# Clean again

In [29]:
components = mesh_surface.split(only_watertight=False)

# 统计数量
print(f"连通组件数量: {len(components)}")

largest = max(components, key=lambda m: len(m.faces))
mesh_clean = largest

# min_faces = 100000
# filtered = [comp for comp in components if len(comp.faces) >= min_faces]

# if len(filtered) > 0:
#     mesh_clean = trimesh.util.concatenate(filtered)
#     mesh_clean.export('cleaned_mesh.obj')
#     print(f"Keep {len(filtered)} component, output new mesh.")
# else:
#     print("All components have been filtered")


v = np.asarray(mesh_clean.vertices)
f = np.asarray(mesh_clean.faces)
n = np.asarray(mesh_clean.vertex_normals)

p = mp.plot(v, f)

mesh_surface = mesh_clean


连通组件数量: 2


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3355654…

# Surface Smoothing

In [30]:
# 将 trimesh 转换为 open3d mesh
mesh_o3d = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(mesh_surface.vertices),
    triangles=o3d.utility.Vector3iVector(mesh_surface.faces)
)

# 可选：计算法线（用于可视化）
mesh_o3d.compute_vertex_normals()

# 执行 Laplacian 平滑
smoothed = mesh_o3d.filter_smooth_laplacian(number_of_iterations=10)

# 可选：重新计算法线
smoothed.compute_vertex_normals()

# 如果你需要导出为 trimesh 继续处理：
smoothed_trimesh = trimesh.Trimesh(
    vertices=np.asarray(smoothed.vertices),
    faces=np.asarray(smoothed.triangles)
)

# ms = pymeshlab.MeshSet()
# ms.add_mesh(pymeshlab.Mesh(mesh_surface.vertices, mesh_surface.faces))


# ms.apply_coord_taubin_smoothing(stepsmoothnum=10, lambda_=0.5, mu=-0.53)


# smoothed_mesh = ms.current_mesh()
# smoothed_trimesh = trimesh.Trimesh(
#     vertices=smoothed_mesh.vertex_matrix(),
#     faces=smoothed_mesh.face_matrix()
# )

In [31]:
v = np.asarray(smoothed.vertices)
f = np.asarray(smoothed.triangles)
# v = np.asarray(smoothed_mesh.vertex_matrix())
# f = np.asarray(smoothed_mesh.face_matrix())

mp.plot(v, f, shading={"wireframe": False})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3367409…